<a href="https://colab.research.google.com/github/babeebird/DeepLearningfromScratch3/blob/main/%EC%A0%9C2%EA%B3%A0%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 제2고지.자연스러운 코드로

- 복잡한 계산도 가능하도록 DeZero를 확장해보자.
- 입력을 여러 개 받는 함수나 출력이 여러 개인 함수도 처리할 수 있도록 DeZero의 기반을 수정하자. 

In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%%time
#- step11.가변 길이 인수(순전파 편)
#- 지금까지는 한 개(길이 1) 데이터의 입력과 출력을 다뤘지만, 이제는 여러개의 데이터를 입출력하고 싶어.
#- 덧셈이나 나눗셈의 경우, 다변수 함수. 입력변수가 여러개.

#- 입력은 리스트, 출력은 튜플 << 왜~?
#- 참고 링크 => https://edykim.com/ko/post/python-list-vs.-tuple/
#- 튜플이 리스트보다 좀 더 메모리효율이 좋고 속도가 빠름
#- 클래스 안에서는 튜플로 빠르게 처리 => 사용자에게는 리스트로 편의성 높임. 
#- 입력값은 같은 종류의 순서가 중요치 않은 데이터 => 리스트, 배열
#- 반면 함수의 작용을 거친 출력값은 입력값에 대해 대응되는 값이기 때문에 순서가 중요 => 튜플 
#- 클래스에서 튜플과 리스트

import numpy as np 


class Variable:
    def __init__(self, data):
        if data is not None: 
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data))) #- 예외처리. data 가 반드시 ndarray 타입이어야 함.

        self.data = data
        self.grad = None
        self.creator = None #- 출력값을 만들어낸 함수. 역전파에 사용. 변수와 함수의 관계 

    def set_creator(self, func):
        self.creator = func #- 해당 Variable 인스턴스의 속성인 self.creator에 func을 할당. 

    def backward(self):
        if self.grad is None: #- 순전파의 최종단계에서는 self.grad가 None 
            self.grad = np.ones_like(self.data) #- self.grad를 1값으로 초기화 하겠다. (self.data와 같은 shape으로)

        funcs = [self.creator] #- funcs는 지역변수.
        while funcs: #- funcs가 빈리스트가 아닌 이상(요소가 한 개이상 있으면/다음으로 진행할 노드(함수)가 있으면) 무한 반복
            f = funcs.pop() 
            x, y = f.input, f.output #- 함수로부터 input과 ouput 값을 꺼내어 할당(unpacking)
            #- 현재 노드(함수)의 입력값인 변수 x. 
            x.grad = f.backward(y.grad) #- y.grad는 upstream grad
            #- x.grad는 dL/dx 
            if x.creator is not None: 
                #- x.creator에 저장된 함수는 변수 x를 출력한 함수
                #- 만약 역전파할 다음 함수가 있다면 funcs 리스트에 추가
                #- 만약 역전파할 다음 함수가 없다면 x는 최초의 입력값이므로 반복문 종료. 
                funcs.append(x.creator)


def as_array(x):
    if np.isscalar(x): #- 스칼라인지 확인해서 boolean 값 반환 
        return np.array(x) #- x가 스칼라값(True)이면 array로 변환하여 반환
    return x #- x가 array(False)이면 그대로 반환 


class Function:
    def __call__(self, inputs): #- inputs는 리스트 타입
        # x = input.data => 바뀌기 전
        xs = [x.data for x in inputs]  # list comprehension. Get data from Variable. inputs의 값들을 리스트 형태로 저장
        # y = self.forward(x) => 바뀌기 전
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]  # list comprehension. Wrap data

        for output in outputs: 
        #- map으로 작성할 수 있지 않을까 ?
        #- map(lambda x: x.set_creator(self), outputs)
        #- cpu time total 711µs 
        #- 1000µs = 1ms 이므로 확실히 map 함수 이용 시 빠르다!
            output.set_creator(self) #- 출력 변수 output의 creator 속성에 현재 함수를 저장 
        self.inputs = inputs #- 함수의 인스턴스 변수 inputs에 입력값 리스트 저장
        self.outputs = outputs #- 함수의 인스턴스 변수 outputs에 출력값 리스트 저장 
        return outputs

    def forward(self, xs): 
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, xs): #- xs는 변수가 두개 담긴 리스트. 
        x0, x1 = xs #- 리스트 xs에서 원소 두개를 꺼내어 x0, x1에 각각 할당. 언패킹(unpacking)
        y = x0 + x1
        return (y,) #- 튜플 반환. 괄호 생략 가능
        #- 함수에서 값을 여러 개 반환할 때에는 값이나 변수를 ,(콤마)로 구분하여 지정함. 

xs = [Variable(np.array(2)), Variable(np.array(3))] #- 여러개의 데이터를 포함한 리스트
f = Add() #- Add() 클래스의 인스턴스, f
ys = f(xs) 
#- 1. xs를 인스턴스 f에 입력으로 넣고 Add() 클래스의 부모클래스인 Function ()이 호출됨.
#- Function() __call__ 함수에서 self는 f 즉 Add()클래스의 인스턴스. 

y = ys[0]
print(y.data)

#- CPU time과 Wall time ? cpu 작동시간 wall time은 끝난시간-시작시간

5
CPU times: user 1.57 ms, sys: 0 ns, total: 1.57 ms
Wall time: 2.45 ms


In [3]:
#- 리스트(list)와 튜플(tuple)
#- 리스트와 튜플은 컨테이너로, 일련의 객체를 저장하는 데 사용한다.  

my_list = [1,2,3]
type(my_list)
my_tuple = (1,2,3)
type(my_tuple)

#- 둘 다 일련의 요소(element)를 갖으며, 요소의 순서를 관리한다. (<=> set, dict)
#- 리스트는 가변적(mutable) 이며, 튜플은 불변적(immutable)이다. (기술적 차이)
my_list[1] = "둘"
my_list.append("넷")
my_list
my_tuple[1] = "둘" #- TypeError : 'tuple' object does not support item assignment
my_tuple.append("넷") #- AttributeError : 'tuple' object has no attribute 'append'

#- 리스트는 단일 종류의 요소를 갖고, 그 일련 요소가 몇 개 들어있는지 명확하지 않을 때 사용.
#- 튜플은 들어 있는 요소의 수를 사전에 정확히 알 때 사용. 
#- 동일한 요소가 들어있는 리스트와 달리 튜플에서는 각 위치의 요소가 중요

list

tuple

[1, '둘', 3, '넷']

TypeError: ignored

In [4]:
#- step 12.가변길이 인수(개선편)
#- 1.Add 클래스를 사용하는 사람을 위해서
#- 2.Add 클래스를 구현하는 사람을 위해서 개선해보자.

#- step 11에서 Add 클래스는 인수를 리스트에 모아서 입력하고, 튜플로 출력값을 반환했음.
#- 리스트, 튜플을 거치지 않고 인수와 결과를 직접 주고받는 것이 사실은 더 자연스럽겠죠 ? 

import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs): #- inputs 앞에 asterisk 애스터리스크(별표)가 붙었다.
    #- 가변인수를 받는 방식 중 하나.
    #- 함수의 *inputs은 부를때는 여러개의 변수로 직접 받지만, 받을때는 input을 리스트로 넣는다.
    #- 리스트를 사용하지 않고, 임의 개수의 인수(가변길이인수)를 입력하여 함수를 호출할 수 있음! 
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) #- 함수 호출 시 애스터리스트를 붙여 리스트 언패킹 
        if not isinstance(ys, tuple): #- 튜플이 아닌 경우 추가 지원 << forward 메서드는 반환 원소가 하나인 경우, 해당 원소 직접 반환  
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0] #- ouputs의 원소가 하나뿐이면 리스트가 아닌 그 원소를 반환
        #- ys[0]를 함수 안에서 처리
    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1): #- 2. Add 클래스를 구현하는 사람을 위한 개선. 입력도 변수를 직접 받고, 출력도 변수를 직접 반환
        y = x0 + x1
        return y #- 반환 원소가 하나인 경우, 해당 원소 직접 반환  
'''
step11에서의 Add class 구현

class Add(Function):
    def forward(self, xs): #- xs는 변수가 두개 담긴 "리스트"
        x0, x1 = xs #- 리스트 xs에서 원소 두개를 꺼내어 x0, x1에 각각 할당. 언패킹(unpacking)
        y = x0 + x1
        return (y,) #- "튜플" 반환. 괄호 생략 가능
        #- 함수에서 값을 여러 개 반환할 때에는 값이나 변수를 ,(콤마)로 구분하여 지정함. 
'''


def add(x0, x1): #- Add 클래스를 파이썬 함수로 사용할 수 있는 코드 추가
#- 이를 통해 Add클래스 생성 과정이 감춰집니다. 
#- y = add(x0, x1)
    return Add()(x0, x1)


x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1) #- 1.사용자를 위한 개선. Function 클래스의 __call__메서드의 가변인자 *inputs 덕분.
print(y.data)

'\nstep11에서의 Add class 구현\n\nclass Add(Function):\n    def forward(self, xs): #- xs는 변수가 두개 담긴 "리스트"\n        x0, x1 = xs #- 리스트 xs에서 원소 두개를 꺼내어 x0, x1에 각각 할당. 언패킹(unpacking)\n        y = x0 + x1\n        return (y,) #- "튜플" 반환. 괄호 생략 가능\n        #- 함수에서 값을 여러 개 반환할 때에는 값이나 변수를 ,(콤마)로 구분하여 지정함. \n'

5


In [5]:
#- Add 클래스를 인수 2개만 더하는 게 아니라, 자유롭게 가변 인수를 받아 더하도록 변경해보자. 
#- 두 개만 더하는게 아니라 여러 개를 더하고 싶을 수 있으니깐!

class Add(Function):
  def forward(self, *xs):
    y = 0
    for x in xs:
      y += x
    return y

def add(*xs):
	return Add()(*xs)

x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
x2 = Variable(np.array(4))
y = add(x0,x1,x2)
print(y.data)

9


- 위치인자와 키워드인자
- 패킹과 언패킹
- 위치인자와 시퀀스 언패킹, 가변인자(*args)
- 키워드 인자와 딕셔너리 언패킹, 키워드 가변인자(**kwargs)

질문: Add 클래스를 인수 2개만 더하는 게 아니라, 자유롭게 가변 인수를 받아 더하도록 만들고 싶어서 아래와 같이 코드를 바꿔봤는데, line4에서 indentationError가 발생했습니다. 왜일까요...?
```
class Add(Function):
	def forward(self, *xs):
  	y = 0
		for x in xs:
			y += x
		return y

def add(*xs):
	return Add()(*xs)

x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
x2 = Variable(np.array(4))
y = add(x0,x1,x2)
print(y.data)
```

들여쓰기 바꾸니 잘 돌아갑니다!


In [6]:
#- list comprehension 과 generator expression
#- list comprehension은 for, in, if 를 적절히 사용하여 list를 쉽고 빠르게 만드는 방법. 
#- generator expression은 list 대신 generator를 반환. 
#- 제너레이터는 모든 값을 메모리에 담고 있지 않고, 필요할 때 값을 생성(generator)해서 반환함. << Lazy evaluation
#- Lazy evaluation이란 계산의 결과값이 필요할 때까지 계산을 늦추는 것.
#- 제너레이터 사용시에는 한번에 한 개의 값만 순환(iterate)할 수 있음. 

x=[n*2 for n in range(10000)] #List Comprehension
y=(n*2 for n in range(10000)) #Generator Expression

type(x) 
#<type 'list'>
print()
type(y)
#<type 'generator'>

list

generator

In [7]:
#- step 13.가변 길이 인수(역전파 편)
#- 복수의 입출력에 대응한 자동미분 구조를 만들어보자.
#- 덧셈의 순전파는 입력이 2개, 출력이 1개인 반면, 역전파는 입력이 1개, 출력이 2개

import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop() #- Variable 인스턴스 
            gys = [output.grad for output in f.outputs] #- 출력 변수인 outputs에 담겨 있는 미분값들을 리스트에 담음
            gxs = f.backward(*gys) #- f의 역전파 메서드를 호출. 이때 gys로 리스트 언팩킹
            if not isinstance(gxs, tuple): #- 위에서 역전파 출력값이 튜플이 아니면
                gxs = (gxs,) #- gxs를 튜플로 변환

            for x, gx in zip(f.inputs, gxs): #- gxs와 f.inputs의 각 원소는 대응관계. 
            #- i번째 원소에 대해 f.inputs[i] 의 미분값은 gxs[i]에 대응함. 

            #- 파이썬 내장함수 zip()의 입력과 출력, 그리고 기능은 ?
            #- input : 여러 개의 순회 가능한(iterable) 객체를 인자로 받고 
            #- output : 각 객체가 담고 있는 원소를 튜플의 형태로 차례로 접근 가능한 반복자(iterator)를 반환함.
            #- 기능 : 옷의 zipper 처럼 두 그룹의 데이터를 서로 엮어준다~ 
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)

'''
class Variable의 기존 역전파 메서드

def backward(self):
    if self.grad is None:
        self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
        f = funcs.pop()
        x, y = f.input, f.output #- 함수의 입력과 출력값을 얻는다.
        x.grad = f.backward(y.grad) #- backward 메서드를 호출한다. 

        if x.creator is not None:
            funcs.append(x.creator)
'''

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) #- 리스트 언팩 << 리스트의 원소를 낱개로 풀어서 지원 
        if not isinstance(ys, tuple): #- 튜플이 아닌 경우 추가 지원
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data #- 수정 전: x = self.input.data > 왜인지 모르겠지만 self.inputs 가 ([1, 2, 3],) 이런 식으로 나와서 [0]을 붙여줘야합니다.
        gx = 2 * x * gy
        return gx


def square(x):
    f = Square()
    return f(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy): #- 덧셈노드의 역전파의 입력으로 gy는 상류에서 흘러오는 값(upstream gradient, global gradient)
        return gy, gy #- gy를 그대로 전파(출력)


def add(x0, x1):
    return Add()(x0, x1)


x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y)) #- z = x^2 + y^2 순전파
z.backward() #- 역전파
print(z.data)
print(x.grad)
print(y.grad)


'\nclass Variable의 기존 역전파 메서드\n\ndef backward(self):\n    if self.grad is None:\n        self.grad = np.ones_like(self.data)\n\n    funcs = [self.creator]\n    while funcs:\n        f = funcs.pop()\n        x, y = f.input, f.output #- 함수의 입력과 출력값을 얻는다.\n        x.grad = f.backward(y.grad) #- backward 메서드를 호출한다. \n\n        if x.creator is not None:\n            funcs.append(x.creator)\n'

13.0
4.0
6.0


In [8]:
#- 파이썬 내장함수 zip() 연습하기

numbers = [1,2,3]
letters = ["ㄱ","ㄴ","ㄷ"]
for i in range(3):
  pair = (numbers[i], letters[i])
  print(pair)

(1, 'ㄱ')
(2, 'ㄴ')
(3, 'ㄷ')


질문: Add 클래스는 forward의 입력이 두개의 변수로 고정되어 있기 때문에 backward의 return이 gy, gy 인걸까요?


여러개 변수를 출력 시 코드 ??? 

backward 메소드 2가지 정리

1. Variable의 backward
- xx.backward() << 인자로 아무것도 전달하지 않는다. 한번 사용하면 처음부터 끝까지 미분 
2. Function의 backward
- xx.backward(x) << 인자를 전달한다. 어떠한 함수 하나만 미분

In [9]:
#- step 14.같은 변수 반복 사용
#- 동일한 변수를 사용하여 덧셈할 경우, 미분 시 오류 발생.
#- 왜? Variable 클래스의 메서드로 backward()를 진행하고 있음. 
#- 따라서 같은 변수를 반복하여 사용하면, 전파되는 미분값이 덮어 쓰여짐.

import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def cleargrad(self): #- 미분값을 초기화(재설정) pg.125
    #- 여러가지 미분을 연달아 계산할 때 똑같은 변수를 재사용 할 수 있게됨.
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop() #- 주목! 15-2. 다음에 처리할 함수를 그 리스트 끝에서 꺼냄 
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
              if x.grad is None: #- 미분값을 처음 설정하는 경우에는
                  x.grad = gx #- 출력 쪽에서 전해지는 미분값을 그대로 대입
              else: #- 그 다음부터는
                  x.grad = x.grad + gx #- 전달된 미분값을 '더해줌'
                  #- 여기서 대입연산자를 사용하여 x.grad += gx 라고 작성하는 경우 문제가 발생한다는데, 부록 A 참고??
                  #- pg.525
                  #- x += x 는 두개 x의 id가 동일/ x = x + x 는 id가 다른 객체가 만들어짐.
                  #- 메모리 효율측면에서는 += 가 유리하지만 dezero에서는 별도의 객체 필요. 
                  #- 그런데.. 요렇게 하면 같은 변수를 사용하여 "다른"계산을 할 경우, 미분값이 누적되어 버림.
                  #- 해결책 : x.cleargrad()
            '''
            for x, gx in zip(f.inputs, gxs): #- gxs와 f.inputs의 각 원소는 대응관계. 
            #- i번째 원소에 대해 f.inputs[i] 의 미분값은 gxs[i]에 대응함. 
                x.grad = gx #- 여기서 문제 발생 !!! 
              '''

            if x.creator is not None:
                funcs.append(x.creator) #- 주목! 15-1. 처리할 함수의 후보를 funcs 리스트의 끝에 추가


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)

#- 첫번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

#- 두번째 계산
#- x = Variable(np.array(3.0))
x.cleargrad() #- 최적화 문제(함수의 최소,최댓값을 찾는 문제) 풀이 시 유용. 28단계에서 사용! 
y = add(add(x, x), x)
y.backward()
print(x.grad)


2.0
1.0


- 교재 125페이지. cleargrad 실행시키지 않아도 2.0, 3.0 나옵니다.
  - 교재에는 2.0, 5.0
  - 왜 그럴까요???

  (저는 cleargrad 안하면 2.0, 5.0 나옵니다)

In [10]:
#- step 15.복잡한 계산 그래프(이론 편) pg.127 << 책 함께 보기! 
#- 그래프의 '연결된 형태'를 위상(topology)라고 함. 
#- 다양한 위상의 계산 그래프에 대응해보자. 어떤 모양으로 연결된 계산 그래프라도 제대로 미분하는 것이 목표
#- 함수 우선순위 - 함수와 변수의 세대(generation)을 기록하자.
#- 순전파 시 '함수'(부모)가 '변수'(자식)를 만들어내는 과정. => 세대 
#- 역전파 시 세대 수가 큰 쪽부터 처리하면 됨.
#- step 16.복잡한 계산 그래프(구현 편)

import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 #- 세대 수를 기록하는 변수.
        #- 인스턴스 변수 generation은 몇 번째 세대의 함수(혹은 변수인지) 나타내는 변수.
        

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 #- 세대를 기록(부모세대 +1)
        #- set_creator 메서드가 호출될때 부모 함수의 세대보다 1만큼 큰 값을 설정 (pg.136 책 그림 참고)

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set() #- funcs 리스트에 같은 함수를 중복 추가하는 것을 막기 위함. 

        def add_func(f): #- 주목! 새로 추가된 함수
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f) #- 왜 필요? pg131 마지막 그림 참고! 
                funcs.sort(key=lambda x: x.generation) #- 리스트의 원소를 x 라고 할때, x.generation을 키로 사용하여 정렬하라
                #- 우리가 원하는 것은 세대가 가장 큰 함수를 꺼내는 것.
                #- 따라서 정렬할 필요없이 '우선순위 큐'를 이용하는 것이 더 효율적. (heapq 모듈 사용)

        add_func(self.creator)

        while funcs:
            f = funcs.pop() #- 다음에 처리할 함수를 funcs 리스트의 끝에서 꺼내기
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator) #- 수정 전: funcs.append(x.creator)
                    #- 처리할 함수의 후보를 funcs 리스트의 끝에 추가하기


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs]) #- Function의 generation 설정
        #- 입력 변수가 둘 이상이라면 가장 큰 generation의 수를 선택. 
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


In [11]:
#- 우선순위 큐를 이용해보자 (heapq) => https://docs.python.org/ko/3/library/heapq.html

In [12]:
#- step 17.메모리 관리와 순환 참조
#- 성능을 개선해보자
#- 메모리 누수(memory leak) 또는 메모리 부족(out of memory) 문제 발생
#- 특히 신경망에서는 큰 데이터를 다루는 경우가 많기에 메모리 관리가 필요!

#- CPython의 메모리 관리는 두 가지 방식. 
#- 1.참조(reference) 수를 세는 방식 => 참조 카운트
#- 2.세대(generation)를 기준으로 쓸모없어진 객체를 회수(collection) => GC(Garbage Collection)
#-   메모리가 부족해지는 시점에 자동으로 호출됨. 또는 gc.collect() 실행

#- 파이썬 메모리 관리의 기본은 참조 카운트 << 구조가 간단하고 속도도 빠름
#- 모든 객체는 참조 카운트가 0인 상태로 생성되고, 다른 객체가 참조할 때마다 1씩 증가. << 내가 너의 이름을 불렀을때, 너는 나에게로 와 한송이 꽃이 되었다 ... 
#- 반대로 객체에 대한 참조가 끊길 때마다 1만큼 감소하다가 0이 되면 파있너 인터프리터가 회수해가는 것(메모리에서 삭제). 
#- 참조 카운트가 증가하는 경우.
#- 1.대입 연산자를 사용할 때
#- 2.함수에 인수로 전달할 때
#- 3.컨테이너 타입 객체(리스트, 튜플, 클래스 등)에 추가할 때

#- DeZero의 Function 인스턴스는 두 개의 Variable 인스턴스(입력, 출력)을 참조함.
#- 출력 Variable 인스턴스는 창조자(부모)인 Function 인스턴스를 참조함.
#- 이 때문에 Function 인스턴스와 Variable 인스턴스가 순환 참조 관계를 만듦.

import weakref #- 약한 참조 : 다른 객체를 참조하되 참조 카운트는 증가시키지 않는 기능
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 #- 세대를 기록

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref. => 참조된 데이터에 접근하려면 output()
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs] #- 함수는 출력 변수를 약하게 참조
        
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


for i in range(10): #- memory profiler 로 메모리 사용량 측정해볼 수 있음! 
    x = Variable(np.random.randn(10000))  # big data
    y = square(square(square(x)))

In [13]:
class obj:
  pass

def f(x):
  print(x)

a = obj() #변수에 대입: 참조 카운트 1
f(a) # 함수에 전달: 함수 안에서는 참조 카운트 2
# 함수 완료: 빠져나오면 참조 카운트 1
a = None # 대입 해제 : 참조 카운트 0

In [14]:
#- 17.2 참조 카운트 방식의 메모리 관리
#- 파이썬에서는 모든 것이 객체. 클래스, 함수, 인스턴스 모두 객체. 

a = obj() #- a의 참조 카운트 1
b = obj() #- b의 참조 카운트 1
c = obj() #- c의 참조 카운트 1

a.b = b #- b 참조카운트 2. a.b? > class obj에 self.b = None을 생략한 것 아닐까요?
b.c = c #- c 참조카운트 2.

a = b = c = None #- a의 참조 카운트는 0이 됨.

In [15]:
#- 17.3 순환 참조
#- GC가 해결해줌. 하지만 메모리 해제를 GC에 미루는 것은 프로그램의 전체 사용량이 커지는 원인! 
#- DeZero에서는 순환 참조를 만들지 않는게 좋겠죠
#- 그러나 현재 DeZero에서는 순환 참조가 있음. pg147 => weakref 모듈로 해결
#- 변수와 함수 사이의 순환참조. 
#- Function 인스턴스는 두 개의 Variable 인스턴스를 참조함

a = obj()
b = obj()
c = obj()

a.b = b #- a.b?
b.c = c
c.a = a

a = b = c = None #- 메모리에서 삭제되지 않음.

In [16]:
#- 17.4 weakref 모듈

import weakref #- 약한 참조 weak reference << 다른 객체를 참조하되 참조 카운트는 증가시키지 않음. 
import numpy as np

a = np.array([1,2,3]) #- 일반 참조 
b = weakref.ref(a) #- 약한 참조 - 순환참조에서 garbage 쌓이는 것 방지를 위함.

b
print()
b() #- 참조된 데이터에 접근 
print()

a = None 
b #- 파이썬 인터프리터에서는 dead; jupyter notebook 인터프리터는 사용자가 모르는 참조를 추가로 유지하기에 b가 여전히 참조 유지. 

<weakref at 0x7fdac728f590; to 'numpy.ndarray' at 0x7fdac728ff30>

array([1, 2, 3])

<weakref at 0x7fdac728f590; to 'numpy.ndarray' at 0x7fdac728ff30>

In [17]:
#- step 18.메모리 절약 모드
#- 1. 역전파 시 사용하는 메모리양 줄이기. 불필요한 미분 결과를 보관하지 않고 즉시 삭제
#- 2. 역전파가 필요 없는 경우용 모드
#       => prediction 이나 transfer learning할 때?
#       => test를 진행할 때? (책에서는 이 경우를 말하고 있음)

import weakref
import numpy as np
import contextlib #- with 문 컨텍스트를 위한 유틸리티. https://docs.python.org/ko/3/library/contextlib.html#module-contextlib

class Config: #- 설정이나 프로그램의 실행 일부 등을 저장해둔 파일
#- 설정 데이터는 한 군데에만 존재하는 것이 좋기에, Config클래스는 인스턴스화하지 않고 클래스 상태로 이용.
#- "역전파 활성 모드"와 "역전파 비활성 모드" 전환 구조 필요.

# 설정 데이터는 단 한 군데에만 존재하는 게 좋습니다. 그래서 Config 클래스는 인스턴스화하지 않고 '클래스' 상태로 이용합니다. 
#   => init()과 같은 것을 다 빼고 설정하는 용도로만 사용한다.
#   => 설정하는 도구들을 답는 틀(?) 정도로만 사용 한다 

    enable_backprop = True #- True면 역전파 활성 모드 


@contextlib.contextmanager #- 데코레이터 << 문맥을 판단하는 함수 
#- 이 함수는 클래스나 별도의 __enter__()와 __exit__() 메서드를 작성할 필요 없이, 
#- with 문 컨텍스트 관리자를 위한 팩토리 함수를 정의하는 데 사용할 수 있는 데코레이터입니다.
#- 
def using_config(name, value): 
  #- 전처리
    old_value = getattr(Config, name) #Config에 name(str)이라는 attribute가 있으면 가져오기 > old_value = True
    setattr(Config, name, value)
    try:
        yield
    finally:
  #- 후처리
        setattr(Config, name, old_value) #다시 True로


def no_grad(): #- 기울기가 필요 없으면 no_grad 함수 호출
    return using_config('enable_backprop', False)


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False): # retain_grad를 False로 바꿉니다. True면 지금까지처럼 모든 변수가 미분 결과(기울기)를 유지합니다. 
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):  # grad를 저장하는 포문 부분
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:             #retain_grad가 False면 (기본값) 중간 변수의 미분값을 모두 None으로 재설정합니다. 
                for y in f.outputs:
                    y().grad = None  # y is weakref
                    #- 참조 카운트가 0이 됨. 미분값 데이터가 메모리에서 삭제됨. 


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:      #- 역전파를 사용할 경우에만 작동 (역전파에 필요할 것들을 모아놓음)
            self.generation = max([x.generation for x in inputs]) #- 세대는 역전파시 노드를 따라가는 순서 정하는 데 사용
            for output in outputs: 
                output.set_creator(self) #- 계산들의 '연결'을 만듦
            self.inputs = inputs #- inputs라는 인스턴스 변수로 참조 => inputs가 참조하는 변수의 참조 카운트 1 증가.
            #- call 메서드에서 벗어난 후에도 메모리에 생존. 
            #- 인스턴스 변수 inputs는 역전파 계산 시 사용. 모델 학습 시 필요
            #- 그러나 모델 추론 시에는 역전파 x. 오직 순전파 => 중간 계산 결과를 버려서 메모리 사용량 줄일 수 있음. 
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()
print(y.grad, t.grad)  # None None
print(x0.grad, x1.grad)  # 2.0 1.0

#- with문을 사용하면, with 블록에 들어갈때의 처리(전처리)와 빠져나올때의 처리(후처리)를 자동으로 할 수 있음. 
with using_config('enable_backprop', False): #- with 문 안에서만 역전파 비활성 모드로 전환하며, 블록 밖에서는 역전파 활성모드로 전환됨.
    x = Variable(np.array(2.0))
    y = square(x)

with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

None None
2.0 1.0


In [18]:
#- step 19.변수 사용성 개선
import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable: 
  #- Variable은 데이터를 담는 '상자'의 역할. 
  #- 사용자 입장에서는 상자보다 '데이터'가 중요 => 상자를 투명하게 해주는 장치를 만들어보자.
  #- Variable 클래스는 ndarray만을 취급하기로 약속했음.
  #- => Variable 인스턴스를 ndarray 인스턴스처럼 보이게 만들자.
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name #- 변수 명 => step 25,26 에서 계산그래프 시각화
        self.grad = None
        self.creator = None
        self.generation = 0

    # property를 지우고 실행시키니 shape 값이 아니라 함수에 대한 정보만 나옵니다.
    #- property 데코레이터를 통해, shape 메서드를 인스턴스 변수처럼 사용할 수 있게 됨.
    #- property란? 역할과 기능은?
    #- @property, @메서드이름.setter 를 통해 메서드를 속성처럼 사용할 수 있음.

    @property 
    def shape(self): #- 배열의 행렬 모양을 튜플로 반환
      #Variable의 backward, cleargrad 메서드를 @property 없이 쓸 수 있었던 이유는 shape처럼 return 값이 없었기 떄문일까요?
      #- y.backward() 처럼 Variable 클래스의 backward 메서드는 인스턴스.메서드()로 호출.

        return self.data.shape

    @property
    def ndim(self): #- 차원 반환
        return self.data.ndim

    @property
    def size(self): #- 배열 요소의 총 갯수 반환
        return self.data.size

    @property
    def dtype(self): #- 배열 전체의 데이터 타입
        return self.data.dtype

    def __len__(self): 
      #- len은 객체 수를 알려주는 파이썬 표준 함수
      #- 파이썬에 내장된 많은 자료형들에는, 해당하는 자료형에 대한 연산을 정의하는 메소드들이 있음.
      #- ‘Magic method’ 이 메소드들은 메소드의 이름 앞뒤에 ‘__‘(double underscore)가 있음. 
      #- ndarray 인스턴스라면 첫 번째 차원의 원소 수를 반환
      #- __len__ 특수 메서드를 통해 Variable 인스턴스에 대해서도 len 함수를 사용할 수 있음. 
        return len(self.data)

    def __repr__(self): 
      #- representation
      #- repr 함수는 어떤 객체의 ‘출력될 수 있는 표현’(printable representation)을 문자열의 형태로 반환
      # 이거 쓰기 전에는 어떻게 나올까? > 'variable' 없이 나옵니다. 줄바꿈은 똑같음
      #- __str__의 본질적인 목적은 추가적인 가공이나 다른 데이터와 호환될 수 있도록 문자열화 하는 것인 반면,
      #- __repr__은 해당 객체를 인간이 이해할 수 있는 "표현"으로 나타내기 위한 용도
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'
        #- variable([[1 2 3]
        #-           [4 5 6]])

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
x.name = 'x'

print(x.name)
print(x.shape)
print(x)

x
(2, 3)
variable([[1 2 3]
          [4 5 6]])


In [19]:
help(repr)

Help on built-in function repr in module builtins:

repr(obj, /)
    Return the canonical string representation of the object.
    
    For many object types, including most builtins, eval(repr(obj)) == obj.



In [20]:
#- @property, @메서드이름.setter 데코레이터 함수

class Person:
    def __init__(self):
        self.__age = 0
 
    @property
    def age(self):           # getter
        return self.__age
 
    @age.setter
    def age(self, value):    # setter
        self.__age = value
 
james = Person()
james.age = 20      # 인스턴스.속성 형식으로 접근하여 값 저장
print(james.age)    # 인스턴스.속성 형식으로 값을 가져옴

20


In [21]:
#- step 20.연산자 오버로드(1)
#- Variable 인스턴스 a, b 에서 *,+ 와 같은 연산자를 활용하여 y = a*b 로 코딩할 수 있으면 좋겠다.
#- 오버로딩(Overloading) vs 오버라이딩(Overriding) ?


#- 오버로딩은 동일한 이름의 함수를 매개변수에 따라 다른 기능으로 동작할 수 있도록 함.
#- 하나의 메소드에 다형성을 부여하는 것.

#- 오버라이딩은 클래스 상속 시 "부모 Class에서 정의한 메소드를 자식 Class 에서 변경하는 것" 으로, 
#- 부모 Class의 메소드 이름과 기본적인 기능은 그대로 사용하지만, 특정 기능을 바꾸고 싶을 때 사용. 
#- Dezero에서 Add Class(자식)는 Function Class(부모)를 상속받음


import weakref
import numpy as np
import contextlib 


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0
    '''
    def __mul__(self, other): 
      #- 곱셈의 특수 메서드(매직 메서드)
      #- 연산자 *를 사용하면 __mul__ 메서드가 호출됨.
      return mul(self, other)
    '''
    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


class Mul(Function): #- Mul 클래스
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1): #- Mul 클래스를 파이썬 함수로 사용하자
    return Mul()(x0, x1)

print(dir(Variable))

#- 파이썬에서는 함수도 객체이기에 함수 자체를 할당할 수 있음
Variable.__add__ = add
Variable.__mul__ = mul

a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

# y = add(mul(a, b), c) 
y = a * b + c #- +, * 연산자를 사용하고 싶다! 
y.backward()

print(y)
print(a.grad)
print(b.grad)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'backward', 'cleargrad', 'dtype', 'ndim', 'set_creator', 'shape', 'size']
variable(7.0)
2.0
3.0


In [22]:
#- step 21.연산자 오버로드(2) 
#- ndarray 인스턴스와 수치 데이터가 함께 사용할 수 있으면 좋겠다. 
#- 예를 들어 Variable 인스턴스 a가 있을 때, a*np.array(3.0)
#- Variable 인스턴스와 ndarray 인스턴스, int, float까지 함께 사용할 수 있도록 해보자.
import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    __array_priority__ = 200 #- 연산자 우선순위 default 0.0 
    #- 참고 : https://numpy.org/doc/stable/reference/arrays.classes.html#numpy.class.__array_priority__
    #- np.array([2.0]) + x 에서 좌항이 ndarray 우항이 Variable 인스턴스이면
    #- 좌항의 ndarray 인스턴스의 __add__ 메서드가 호출됨. 
    #- 우리는 Variable 인스턴스의 __radd__메서드가 호출되길 원함  

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_variable(obj): #- obj가 Variable이면 그대로 반환, ndarray라면 Variable 인스턴스로 변환하여 반환.
    if isinstance(obj, Variable): # Variable이 type이라면 class와 type은 같은 의미??
        return obj
    return Variable(obj)


def as_array(x): #- x가 float 또는 int 인 경우, ndarray 인스턴스로 변환.
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs] #- as_varaible 함수 사용
        #- DeZero에서 사용하는 모든 함수(연산)은 Function 클래스를 상속하므로, 
        #- 실제 연산은 Function 클래스의 __call__ 메서드에서 이뤄짐

        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    x1 = as_array(x1)       #일일이 함수에 as_array를 적용시킴
    return Add()(x0, x1)    #x0는 왜 as_array를 안했을까 => 굳이 할 필요가 없다?
    


class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)

    #- x * 2.0은 가능하지만, 2.0 * x 도 가능 ??
    #- 2.0 * x 실행 시 오류 과정
    #- 1.연산자 왼쪽에 있는 2.0의 __mul__ 메서드 호출 시도 
    #- 2.하지만 2.0은 float 타입이므로 __mul__ 메서드는 구현되어 있지 않음
    #- 3.다음으로 *연산자 오른쪽에 있는 x의 특수 메서드를 호출하려 시도
    #- 4.x는 오른쪽에 있기에 __rmul__ 메서드를 호출하려 시도
    #- 5.하지만 Variable 인스턴스에는 __rmul__ 메서드가 구현되어 있지 않음 
    #- => rmul 메서드를 구현하면 되겠다! 
    #- __rmul__(self, other) 에서 x-> self, 2.0-> other

Variable.__add__ = add #왼쪽이 무조건 Variable이라는 가정 하에 진행
Variable.__radd__ = add #오른쪽이 무조건 Variable이라는 가정 하에 진행
Variable.__mul__ = mul
Variable.__rmul__ = mul

x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

y = x + 3.0
print(y)

y = 3.0 * x + 1.0
print(y)

#- 21.4 문제점2 : 좌항이 ndarray 인스턴스인 경우. __array_priority__ = 200 가 없으면 어떤 문제가 발생 ? 

variable(5.0)
variable(5.0)
variable(7.0)


In [23]:
 #- step 22.연산자 오버로드 (3)
 #- 왜 굳이 sub, rsub 나눠서 정의 => 오버로드를 하는걸까 ? 
 #- 그냥 sub에 조건문 활용해서 경우에 따라 입력 변수를 변환해준다음에 오버로드하면 안되나 ?  
 
import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    __array_priority__ = 200

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]

        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)


class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)


class Neg(Function): #- 1.Neg 클래스, 음수(부호변환)
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy


def neg(x): #- 2.neg 파이썬 함수
    return Neg()(x)


class Sub(Function): #- 뺄셈
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy


def sub(x0, x1):
    x1 = as_array(x1) 
    #- x1은 scalar(int,float) -> ndarray(as_array()함수) -> Variable(as_variable()함수)
    #- 반면 x0이 Variable 인스턴스가 아닌 경우 에러
    #- => rsub 메서드 정의 하여 인수의 순서를 바꿔서 호출하자.
    return Sub()(x0, x1) 


def rsub(x0, x1): 
  #- 특수메서드 __rsub__ 에 인수가 전달되는 방식은 cross! 
  #- 예를 들어 2.0 - x 의 경우, __rsub__(self, other)에서 x->self(x0), 2.0->other(x1)
    x1 = as_array(x1)
    return sub(x1, x0) #- x1 - x0 이므로


class Div(Function): #- 나눗셈
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy): #- 나눗셈의 미분
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1 
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1


def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)


def rdiv(x0, x1):
    x1 = as_array(x1)
    return div(x1, x0)


class Pow(Function): #- 거듭제곱 
    def __init__(self, c): #- c는 지수, 상수로 취급하겠음
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy): 
        x = self.inputs[0].data
        c = self.c

        gx = c * x ** (c - 1) * gy
        return gx


def pow(x, c):
    return Pow(c)(x)


Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul
Variable.__neg__ = neg #- 3.오버로딩
Variable.__sub__ = sub
Variable.__rsub__ = rsub
Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv
Variable.__pow__ = pow

x = Variable(np.array(2.0))
y = -x
print(y)  # variable(-2.0)

y1 = 2.0 - x
y2 = x - 1.0
print(y1)  # variable(0.0)
print(y2)  # variable(1.0)

y = 3.0 / x
print(y)  # variable(1.5)

y = x ** 3
y.backward()
print(y)  # variable(8.0)

variable(-2.0)
variable(0.0)
variable(1.0)
variable(1.5)
variable(8.0)


__init__.py 덕분에 .core_simple을 안 써도 된다. 왜??

In [26]:
#- step 23

# Add import path for the dezero directory.
'''
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))

import numpy as np
from dezero import Variable


x = Variable(np.array(1.0))
y = (x + 3) ** 2
y.backward()

print(y)
print(x.grad)
'''

"\nif '__file__' in globals():\n    import os, sys\n    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))\n\nimport numpy as np\nfrom dezero import Variable\n\n\nx = Variable(np.array(1.0))\ny = (x + 3) ** 2\ny.backward()\n\nprint(y)\nprint(x.grad)\n"

- 모듈: import하여 사용하는 것을 가정하고 만든 파이썬 파일(.py)
- 패키지: 여러 모듈을 묶은 것. 디렉터리 안에 모듈 추가
- 라이브러리: 여러 디렉터리(패키지)를 묶은 것

우리는 dezero라는 패키지에 모듈을 추가할 것이다

In [27]:
#- step 24. 복잡한 함수의 미분 
#- 라이브러리와 프레임워크의 차이는 ??? 
'''
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
import numpy as np
from dezero import Variable


def sphere(x, y):
    z = x ** 2 + y ** 2
    return z


def matyas(x, y):
    z = 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y
    return z


def goldstein(x, y):
    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * \
        (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z


x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)  # sphere(x, y) / matyas(x, y)
z.backward()
print(x.grad, y.grad)
'''

"\nif '__file__' in globals():\n    import os, sys\n    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))\nimport numpy as np\nfrom dezero import Variable\n\n\ndef sphere(x, y):\n    z = x ** 2 + y ** 2\n    return z\n\n\ndef matyas(x, y):\n    z = 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y\n    return z\n\n\ndef goldstein(x, y):\n    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) *         (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))\n    return z\n\n\nx = Variable(np.array(1.0))\ny = Variable(np.array(1.0))\nz = goldstein(x, y)  # sphere(x, y) / matyas(x, y)\nz.backward()\nprint(x.grad, y.grad)\n"

##Define and Run (정적 그래프 방식)
- 계산 그래프를 정의한 다음 데이터를 흘려보낸다
- 계산 그래프는 사용자가 제공, 프레임워크는 주어진 그래프를 컴퓨터가 처리할 수 있는 형태로 변환(컴파일)하여 데이터를 흘려보내는 식


---


- !! 중요한점 !! '계산 그래프 정의'와 '데이터 흘려보내기'가 분리되어 있다는 것


---



```
a = Variable('a')
b = Variable('b')
c = a * b
d = c + Constant(1)

#계산 그래프 컴파일
f = compile(d)

#데이터 흘려보내기
d = f(a = np.array(2), b = np.array(3))

```

- 여기서 실제 계산이 이루어지지 않는다 (print(d)해도 안뜰 걸?)
- 실제 '수치'가 아닌 '기호'를 대상으로 프로그래밍되었기 때문
- 이렇게 실제 데이터가 아닌 기호를 사용한 추상적인 계산 절차를 코딩해야 한다

---

- 또한 도메인 특화 언어(DSL)을 사용해야 한다
- 도메인 특화 언어는 '파이썬 위에서 동작하는 새로운 프로그래밍 언어'라고 할 수 있다
- 미분을 위해 설계된 언어이기도 하다


---


- 대표적으로 tensorflow, caffe, CNTK?
- tensorflow는 2.0부터 Define-by-Run 추가


## Define-by-Run(동적 계산 그래프 방식)
- 데이터를 흘려보냄으로써 계산 그래프가 정의된다
- 즉 '데이터 흘려보내기'와 '계산 그래프 구축'이 동시에 이루어진다

---

- DeZero의 경우 데이터를 흘려보낼 때 자동으로 계산 그래프를 구성하는 '연결(참조)'을 만든다
- 이 연결이 계산 그래프에 해당

---
- 넘파이를 사용하는 일반적인 프로그래밍과 똑같은 형태로 코딩 가능



```
import numpy as np
from dezero import Variable

a = Variable(np.ones(10))
b = Variable(np.ones(10) * 2)
c = b * a
d = c + 1
print(d)

```
- 유일한 차이는 넘파이 데이터를 Variable 클래스로 감쌌다는 점

---

- 2015년 체이너에 의해 처음 제창
- 대표적으로 파이토치, MXNet, DyNet, tensorflow(2.0 이상)

---

- 디버깅에 유리(pdb 같은 파이썬 디버거 사용 가능)
